In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [2]:
# csvファイルからPandas DataFrameへ読み込み
train = pd.read_csv('train.csv', delimiter=',', low_memory=False)
submit_data = pd.read_csv('test.csv', delimiter=',', low_memory=False)

In [3]:
#冒頭を表示
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,...,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,...,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,...,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Class_2


In [4]:
#データの形を確認
train.shape

(200000, 77)

In [5]:
#データのタイプを確認
train.dtypes

id             int64
feature_0      int64
feature_1      int64
feature_2      int64
feature_3      int64
               ...  
feature_71     int64
feature_72     int64
feature_73     int64
feature_74     int64
target        object
Length: 77, dtype: object

In [6]:
#trainの基本的統計量を表示
train.describe(include='all').transpose()

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,200000,NaN,NaN,NaN,99999.5,57735.2,0,49999.8,99999.5,149999,199999
feature_0,200000,NaN,NaN,NaN,0.97271,3.94184,0,0,0,1,61
feature_1,200000,NaN,NaN,NaN,1.16837,3.99341,0,0,0,1,51
feature_2,200000,NaN,NaN,NaN,2.21932,6.47657,0,0,0,1,64
feature_3,200000,NaN,NaN,NaN,2.29673,7.55186,0,0,0,1,70
...,...,...,...,...,...,...,...,...,...,...,...
feature_71,200000,NaN,NaN,NaN,0.806895,2.45874,0,0,0,1,30
feature_72,200000,NaN,NaN,NaN,1.28293,4.26142,0,0,0,1,61
feature_73,200000,NaN,NaN,NaN,2.94021,10.7847,0,0,0,1,130
feature_74,200000,NaN,NaN,NaN,0.632005,3.92531,0,0,0,0,52


In [7]:
#trainのtargetをカテゴリーに変換
train.target = train.target.astype('category')

train.dtypes

id               int64
feature_0        int64
feature_1        int64
feature_2        int64
feature_3        int64
                ...   
feature_71       int64
feature_72       int64
feature_73       int64
feature_74       int64
target        category
Length: 77, dtype: object

In [8]:
# ラベルエンコーディング（LabelEncoder）
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
encoded = le.fit_transform(train.target.values)
decoded = le.inverse_transform(encoded)
train.target = encoded

#冒頭を表示して確認
train.target.head()

0    5
1    5
2    1
3    7
4    1
Name: target, dtype: int64

In [9]:
#訓練データを分割する
train_x, train_y = train.drop(['target'], axis=1).drop(['id'], axis=1), train.target

In [10]:
#trainデータを学習データと検証データに分割する
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3)

#冒頭を表示して確認
x_train.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
171802,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,3,0,0,0,0,0
164738,0,0,1,0,0,2,1,6,0,0,...,0,0,0,0,8,0,0,0,0,0
88171,0,0,0,0,0,0,0,0,9,0,...,0,0,0,0,2,1,0,0,0,0
146092,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70047,0,1,0,0,0,0,0,1,0,0,...,0,3,0,0,1,0,0,0,0,0


In [11]:
#モデルを構築
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(75,)),
    keras.layers.Dense(282, activation='relu'),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(121, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(131, activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(9, activation='softmax')
])

In [12]:
#モデルをコンパイルする
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
#学習させる
model.fit(x_train, y_train, batch_size=960, epochs=464)

Epoch 1/464
146/146 [==============================] - 2s 9ms/step - loss: 2.2383 - accuracy: 0.2456
Epoch 2/464
146/146 [==============================] - 2s 13ms/step - loss: 1.8789 - accuracy: 0.3102
Epoch 3/464
146/146 [==============================] - 2s 13ms/step - loss: 1.8294 - accuracy: 0.3286
Epoch 4/464
146/146 [==============================] - 1s 9ms/step - loss: 1.8104 - accuracy: 0.3351
Epoch 5/464
146/146 [==============================] - 1s 8ms/step - loss: 1.8019 - accuracy: 0.3394
Epoch 6/464
146/146 [==============================] - 1s 8ms/step - loss: 1.7910 - accuracy: 0.3443
Epoch 7/464
146/146 [==============================] - 1s 9ms/step - loss: 1.7838 - accuracy: 0.3467
Epoch 8/464
146/146 [==============================] - 1s 8ms/step - loss: 1.7871 - accuracy: 0.3461
Epoch 9/464
146/146 [==============================] - 1s 7ms/step - loss: 1.7808 - accuracy: 0.3502
Epoch 10/464
146/146 [==============================] - 1s 8ms/step - loss: 1.7795 - accu

146/146 [==============================] - 1s 9ms/step - loss: 1.6971 - accuracy: 0.3851
Epoch 82/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6940 - accuracy: 0.3856
Epoch 83/464
146/146 [==============================] - 2s 10ms/step - loss: 1.6963 - accuracy: 0.3843
Epoch 84/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6917 - accuracy: 0.3850
Epoch 85/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6915 - accuracy: 0.3863
Epoch 86/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6885 - accuracy: 0.3854
Epoch 87/464
146/146 [==============================] - 2s 10ms/step - loss: 1.6919 - accuracy: 0.3874
Epoch 88/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6930 - accuracy: 0.3842
Epoch 89/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6928 - accuracy: 0.3856
Epoch 90/464
146/146 [==============================] - 1s 7ms/step - loss: 1.6877 - accurac

146/146 [==============================] - 1s 8ms/step - loss: 1.6704 - accuracy: 0.3949
Epoch 161/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6727 - accuracy: 0.3909
Epoch 162/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6668 - accuracy: 0.3975
Epoch 163/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6654 - accuracy: 0.3946
Epoch 164/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6724 - accuracy: 0.3938
Epoch 165/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6695 - accuracy: 0.3950
Epoch 166/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6642 - accuracy: 0.3975
Epoch 167/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6699 - accuracy: 0.3956
Epoch 168/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6749 - accuracy: 0.3925
Epoch 169/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6659 - 

146/146 [==============================] - 2s 10ms/step - loss: 1.6629 - accuracy: 0.3954
Epoch 240/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6586 - accuracy: 0.3989
Epoch 241/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6567 - accuracy: 0.3987
Epoch 242/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6578 - accuracy: 0.3989
Epoch 243/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6556 - accuracy: 0.3990
Epoch 244/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6608 - accuracy: 0.3990
Epoch 245/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6571 - accuracy: 0.4007
Epoch 246/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6544 - accuracy: 0.4011
Epoch 247/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6581 - accuracy: 0.3985
Epoch 248/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6543 

146/146 [==============================] - 1s 8ms/step - loss: 1.6482 - accuracy: 0.4018
Epoch 319/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6532 - accuracy: 0.4006
Epoch 320/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6538 - accuracy: 0.4007
Epoch 321/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6455 - accuracy: 0.4026
Epoch 322/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6488 - accuracy: 0.4022
Epoch 323/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6556 - accuracy: 0.4002
Epoch 324/464
146/146 [==============================] - 1s 8ms/step - loss: 1.6501 - accuracy: 0.4026
Epoch 325/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6506 - accuracy: 0.4013
Epoch 326/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6504 - accuracy: 0.4022
Epoch 327/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6491 - 

146/146 [==============================] - 1s 10ms/step - loss: 1.6472 - accuracy: 0.4030
Epoch 398/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6494 - accuracy: 0.4006
Epoch 399/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6443 - accuracy: 0.4038
Epoch 400/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6477 - accuracy: 0.4020
Epoch 401/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6453 - accuracy: 0.4048
Epoch 402/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6453 - accuracy: 0.4041
Epoch 403/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6429 - accuracy: 0.4045
Epoch 404/464
146/146 [==============================] - 2s 11ms/step - loss: 1.6483 - accuracy: 0.4031
Epoch 405/464
146/146 [==============================] - 1s 9ms/step - loss: 1.6421 - accuracy: 0.4070
Epoch 406/464
146/146 [==============================] - 1s 10ms/step - loss: 1.6

In [14]:
# 検証データへの精度
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

1875/1875 - 1s - loss: 1.8120 - accuracy: 0.3431

Test accuracy: 0.34308332204818726
